#  李顺立\_2018210765\_L6
----
----

## 构建文本词频矩阵并尝试 ***线性回归*** 
* 导入相应模块
* 数据预处理
    * 定义mac打开zip文件函数
    * 加载本地字典
    * 加载停用词表
    * 定义寻找薪资的函数
    * 定义职位和薪资要求区分函数
* 文本词频处理
     + 搭建词频数据框
     + 构建线性回归模型
     + 利用*sklearn* 中的*linear_model*做回归
* 总结
----

## 导入相应模块

In [1]:
###导入模块
import jieba   #jieba分词
import jieba.analyse
import re
import numpy as np
import pandas as pd
from sklearn import linear_model   #线性回归
from zipfile import ZipFile  

----
## 数据预处理
### 定义mac打开zip文件函数

In [2]:
def open_zip(name):
    '''用于mac打开zip文件，返回打开后的文件，用于txt读取'''

    my_zip = ZipFile(name, 'r')  #打开压缩包
    zip_file  = [my_zip.read(i).decode('utf-8') for i in my_zip.namelist()
                         if '__M' not in i.encode('cp437').decode('utf-8') 
                         and my_zip.read(i).decode('utf-8') is not '']
    my_zip.close()   ## 打开文本文件，使用‘utf-8’编码和‘cp437’解码并且保证结果中不含空的list
    return zip_file

In [3]:
job_requirements = open_zip('职位要求.zip')  #打开职位要求zip中的所有职位的要求
print(job_requirements)

['\n\n薪资：2万-4万\n北京3-5年硕士招3人岗位职责：\n1、负责将反映业界高平的语音识别，语音合成，语音唤醒，声纹识别等语音技术，工程落地到车载 操作系统的语音模块中，为车载操作系统提供核心语音能力；\n2、 负责认知语音识别引擎，语音数据平台，语音训练平台实现，搭建，测试及维护；\n3、负责语音识别相关算法的研究与开发，声学模型训练、语言模型训练的算法研究及解码器技术研发；\n4、负责跟进行业前沿语音技术发展趋势，跟踪国际较新算法发展方向和相关技术；\n任职资格：\n1.计算机科学，软件工程，电子，数学，自动化等相关专业，硕士（含）以上；\n2. 熟悉语音识别，语音唤醒，语音合成，声纹识别技术， 具备扎实的模式识别、机器学习和深度学习等知识基础和丰富的项目研发经验；\n3、熟练掌握Kaldi，HTK，TensorFlow等社区开源工具中的一种及以上\n4、精通C/C++，Python，Shell编程语言，对数据结构和算法设计有深刻理解\n5、有优秀的解决问题能力和面对困难锲而不舍的自驱力。有优秀的团队合作能力者\n6、具备语音识别，语音唤醒，语音合成领域行业领先企业的工作经验者优先\n7、在相关国际会议或主流期刊上发表论文者优先（ICASSP，Interspeech，ACL，ICML，NIPS）', '\n薪资：1.3万-1.8万\n北京3-5年本科招1人\n\n岗位职责\n\n1、从事二手互联网行业的市场研究，分析行业及业务现状，预估行业及业务未来的发展；\n2、参与年度/季度预算编制，并定期进行数据预实对比分析，形成PPT等展示报告；\n3、主动搜集业务数据，进行分析，制作数据分析报告模板，进行数据整合分析，定期形成业财分析报告；\n4、及时汇报企业经营状况、财务收支及各项财务计划的具体执行情况，为企业决策层提供财务分析与预测报告，并提出支持性的建议；\n5、完成上级交办的其他工作。\n任职要求\n\n1、本科及以上学历，统计、财务、计算机、数学、市场相关背景，2年及以上财务数据分析工作经验，了解并热爱汽车或互联网行业，有相关工作经验者优先；\n2、熟悉定性或定量分析方法，熟练掌握各类数据研究方法与模型，能够出色完成研究报告的撰写； \n3、对互联网领域富有激情，较强的商业意识和数据思维，优秀的商业分析能力和逻辑思维能力；\n4、对数据敏感，具备较

In [4]:
data = job_requirements.copy()   #数据拷贝
len(data)

16

### 加载本地词典

In [5]:
jieba.load_userdict('/anaconda3/lib/python3.7/site-packages/jieba/dict.txt')

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/_4/0d6f0wv952s2rc4jhfhtzstw0000gn/T/jieba.cache
Loading model cost 0.675 seconds.
Prefix dict has been built succesfully.


In [6]:
jieba.load_userdict('my_dict.txt')   #自定义用户词典

### 加载停词表

In [7]:
with open('中文停用词.txt', 'r') as file:
    my_stopwords = file.read()  #感觉没用？

### 定义寻找薪资的函数

In [8]:
def Find_Money(salary):
    '''
    找到一个工资里面的数据，并转换为元
    有可能有千，万，k，w
    salary示例：薪资：20k-21千
    返回单位为元的数据：示例[20000.0, 21000.0]
    '''
    
    punc = r'\d+.\d+.*?千|\d+.\d+.*?万|\d+.\d+.*?k|\d+.\d+.*?w|\d+.*?k|\d+.*?w|\d+.*?千|\d+.*?万' 
    #找到钱的各种可能组合
    result = re.findall(punc, salary)  #找到所有钱的组合
    yuan = []
    for item in result:
        digit = re.findall(r'\d+.\d+|\d+', item)[0]   #找到所有数值
        if 'k' in item:
            yuan.append(float(digit)*1000)
        elif '千' in item:
            yuan.append(float(digit)*1000)
        else:
            yuan.append(float(digit)*10000) 
    return yuan

In [9]:
Find_Money('薪资：13千 -1.8万')

[13000.0, 18000.0]

### 定义职位和薪资要求区分函数

In [10]:
def Salary_Require(data, top = 10):
    '''data是每一条含有薪资的职位要求'''
    '''返回薪资的平均值和前top个职位要求, 默认10'''
    data_cut = []  #将薪资和分词结果放在一行
    for item in data:
        salary_str =re.findall(r'薪资.*?\n', item)[0]  #提取后的薪资str，示例：‘薪资8千-5万’
        salary = np.mean(Find_Money(salary_str))  #对薪资去均值
        require_cut = jieba.analyse.extract_tags(item.lower(), topK = top)   #分词结果提取前top个，用于筛选
        data_cut.append([salary, require_cut])   #将结果放在data_cut中
    return data_cut

In [11]:
Salary_Require(data, 4)

[[30000.0, ['语音识别', '语音合成', '语音唤醒', '语音']],
 [15500.0, ['分析', '数据分析', 'ppt', '工作经验']],
 [7000.0, ['项目', '客户', '撰写', '数据']],
 [22500.0, ['机器学习', '贝叶斯', '算法', '监测']],
 [12000.0, ['本科', '业务', '优先', '1.5']],
 [14500.0, ['算法', '算法研究', 'go', '开发']],
 [24000.0, ['机器学习', '用户', '系统', '熟悉']],
 [20000.0, ['算法', '检测', '具有', '能力']],
 [25000.0, ['nlp', '对话', '硕士', '算法']],
 [22500.0, ['电力', '负责', '电子产品', '控制']],
 [20000.0, ['算法', '图像处理', '检测', '缺陷']],
 [11500.0, ['数据分析', '数据', '本科', '精通']],
 [17500.0, ['研究', '有过', '技术', '环境参数']],
 [27500.0, ['机器学习', 'qeexo', '产品', '硕士']],
 [19000.0, ['算法', '图像处理', '研发', '语言']],
 [12500.0, ['运营', '优化', '数据', '负责']]]

----
## 文本词频处理
### 搭建词频数据框

In [12]:
column_name = ['salary', '机器学习', '硕士', 'python', 'sql', '算法', '数据分析']  #手动输入自变量

In [13]:
#定义词语搜索函数
def Search(strr, item):
    '''判断字符串strr是否在item中，如果在返回1，不在返回0'''
    if strr in item:
        return 1
    else:
        return 0

In [14]:
Search('ss', ['s', 'qq'])

0

In [15]:
word_frequence = pd.DataFrame(columns = column_name)  #构建一个空的DataFrame
salary_require = Salary_Require(data, 50)    #找出薪资数据和分词结果（50个）

for i in range(len(salary_require)):
    df = []   #定义一个空的df用于记录每一行的数据
    for item in column_name:  #按照所定义的几个重要的变量一次查看是否有该变量
        if item == 'salary':  #如果是工资，则直接将工资放在这一列
            df.append(salary_require[i][0])   #i：第i个职位， 0：第i个职位的薪资（取了均值）
        else:
            df.append(Search(item, salary_require[i][1]))  #如果不是工资，则将结果0，1返回到df中
    word_frequence.loc[i] = df   ##将做一次的结果放在dataframe中

In [16]:
word_frequence

,salary,机器学习,硕士,python,sql,算法,数据分析
0,30000.0,1.0,1.0,1.0,0.0,1.0,0.0
1,15500.0,0.0,0.0,0.0,0.0,0.0,1.0
2,7000.0,0.0,0.0,1.0,1.0,0.0,1.0
3,22500.0,1.0,0.0,1.0,0.0,1.0,1.0
4,12000.0,0.0,0.0,1.0,1.0,0.0,0.0
5,14500.0,0.0,1.0,1.0,0.0,1.0,0.0
6,24000.0,1.0,0.0,1.0,0.0,1.0,0.0
7,20000.0,0.0,0.0,0.0,0.0,1.0,0.0
8,25000.0,1.0,1.0,1.0,0.0,1.0,0.0
9,22500.0,0.0,1.0,0.0,0.0,0.0,0.0


### 构建线性回归模型

In [17]:
#利用最小二乘法自定义一个线性回归函数
class linear_regression(object):
    '''
    使用最小二乘法计算线性回归的系数:
    fit(X, y, intercept = True)模型搭建，传回系数
    X：自变量数据，array格式
    y：因变量数据，array格式，行向量
    intercept：截距项，默认True，有截距
    
    predict(X)模型预测值
    X：预测值自变量 array格式
    '''
    def __init__(self): #self class自身变量，有两个属性：beta和intercept
        self.beta = None
        self.intercept = None  #是否有截距

#定义回归模型函数
    def fit(self, X, y, intercept = True):
        self.intercept = intercept  #是否有截距项
        
        if self.intercept == True: #如果为False,则X不作处理
            X = np.insert(X, 0, 1, axis=1)   #第0列全部插入1， 默认是有截距项的
            print('模型中有截距项！')
        else:
            print('模型中无截距项！')
        
        XTX_1 = np.linalg.inv(X.T.dot(X))  #求XTX的逆
        self.beta = XTX_1.dot(X.T).dot(y)  #得到了系数beta
        
        return self.beta  #fit后得到系数
        
##定义预测函数：
    def predict(self, X):  #输入X,注意X是个np.array的行向量
        
        if self.intercept == True: #如果为False,则X不作处理
            X = np.insert(X, 0, 1, axis=1)   #第0列全部插入1
        y_pred = X.dot(self.beta)
        
        return y_pred

In [18]:
# 利用自定义回归函数进行拟合
X_train = np.array(word_frequence.iloc[:, 1:])  #训练集
y_train = np.array(word_frequence.iloc[:,0])
print(X_train, '\n')
print(y_train)

[[1. 1. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 1. 0. 1.]
 [1. 0. 1. 0. 1. 1.]
 [0. 0. 1. 1. 0. 0.]
 [0. 1. 1. 0. 1. 0.]
 [1. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 [1. 1. 1. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 1. 0.]
 [1. 1. 1. 0. 1. 0.]
 [1. 1. 1. 0. 1. 0.]
 [0. 0. 0. 1. 0. 1.]] 

[30000. 15500.  7000. 22500. 12000. 14500. 24000. 20000. 25000. 22500.
 20000. 11500. 17500. 27500. 19000. 12500.]


In [19]:
#模型fit
my_linear_model = linear_regression()
my_linear_model.fit(X_train, y_train)

模型中有截距项！


array([19571.42857143, 11383.11688312,   577.92207792, -5136.36363636,
       -2383.11688312,  -720.77922078, -4896.1038961 ])

In [20]:
###做个预测
X_pre = np.array([1, 1, 1, 1, 1, 1]).reshape(1, -1)  #输入的值是一个行向量
my_linear_model.predict(X_pre)

array([18396.1038961])

### 利用$sklearn$中的$linear\_model$做回归

In [21]:
X = word_frequence.iloc[:, 1:] #定义数据
y = word_frequence.iloc[:, 0]
#print(X, '\n', y)

In [22]:
lm = linear_model.LinearRegression(fit_intercept = True, normalize = False) #有截距，数据不做标准化
result = lm.fit(X,y)

In [23]:
result.score(X,y)   #查看R^2

0.842844159859516

In [24]:
result.coef_  #查看自变量系数

array([11383.11688312,   577.92207792, -5136.36363636, -2383.11688312,
        -720.77922078, -4896.1038961 ])

In [25]:
result.intercept_  #查看截距项

19571.42857142857

In [26]:
#预测
X0 = ([[1,1,1,1,1,1]])
lm.predict(X0)

array([18396.1038961])

----
## 总结

* 通过线性回归结果可以看出，回归的$R^2$=0.84，模型拟合就好。但是，观察所选变量的系数，python、sql、算法和数据分析的系数均为负数，显然不符合意义。
* 可能是由于数据太少，只有16个观测值导致的，所以后续还需要继续改进。